In [ ]:
%pylab inline

import sys
import numpy as np
import Nio
from mpl_toolkits.basemap import Basemap
import matplotlib.pyplot as plt
from IPython.html.widgets import interact

sys.path.append('../pystormtracker/')
from detector import Center, RectGrid
from linker import Tracks

try:
    import cPickle as pickle
except ImportError:
    import pickle

In [18]:
tracks = pickle.load(open("../pystormtracker/tracks.pickle", "rb"))
# tracks = pickle.load(open("../pystormtracker/tracks_cesm.pickle", "rb"))

In [ ]:
m = Basemap(projection='cyl',llcrnrlat=-90,urcrnrlat=90,\
            llcrnrlon=0,urcrnrlon=360,resolution='c')
fig = plt.figure(figsize=(16,10))
ax = fig.add_axes([0.1,0.1,0.9,0.9])
m.drawcoastlines()

counter = 0
for t in tracks:
    lats = []
    lons = []
    
    if t[0].abs_dist(t[-1]) >= 1000. and len(t) >= 8:
        lats = [c.lat for c in t]
        lons = [c.lon for c in t]
            
        # lons = [l-360. if l>180. else l for l in lons]
        
        # Trick to remove the horizontal streaks
        # http://pythology.blogspot.com/2014/02/how-to-avoid-jumping-line-plots-in.html
        segment = np.vstack( (lons,lats))
        bleft = 0.
        bright = 360.
        segment[0,segment[0]> bright] -= 360.                                                                 
        segment[0,segment[0]< bleft]  += 360.
        threshold = 90.                                                                                      
        isplit = np.nonzero(np.abs(np.diff(segment[0])) > threshold)[0]                                                                                         
        subsegs = np.split(segment,isplit+1,axis=+1)

        for seg in subsegs:
            x,y = m(seg[0], seg[1])
            m.plot(x, y, linewidth=1, color='b')
        
        counter = counter + 1
        
plt.show()

print(counter)